In [11]:
from collections import Counter, defaultdict
from nltk.tokenize import word_tokenize
import pickle
import string
import nltk
import re
from nltk.tokenize import word_tokenize
from searchEngine import MySearchEngine

In [12]:
class entityDatabase:
    def __init__(self, pickle_path):
        self.ent_dict = defaultdict(list)
        pfile = open(pickle_path, mode='rb')
        p = pickle.load(pfile)
        #pickle_path example: "C:\\Users\\User\\Desktop\\beaver\\NumberWon\\numberwon\\entity\\test.pickle"
        self.entize(p)
        self.engine = MySearchEngine(file=pickle_path,version='d')
        
    def get_by_id(self, id):
        return self.ent_dict[id]
    
    def entize(self, pickle):
        for key, val in pickle.items():
            tokens = nltk.word_tokenize(val)
            pos = nltk.pos_tag(tokens)
            named_entities = nltk.ne_chunk(pos, binary = True)
            for i in range(0, len(named_entities)):
                ents = named_entities.pop()
                if getattr(ents, 'label', None) != None and ents.label() == "NE":
                    z = list(zip(*[ne for ne in ents]))[0]
                    z = " ".join(z)
                    self.ent_dict[key].append(z)

    #fix!!!             
    def searchNentity(self, qword):
        topdoc = self.engine.query(qword)[0][0]
        return self.top_entity_associated_with_item(qword,self.engine.raw_text[topdoc]) #Megan's method

    def docsearch(self, qword):
        topdoc = self.engine.query(qword)[0][0]
        raw = self.engine.raw_text[topdoc] #whole doc
        return re.match(r'(?:[^.:;]+[.:;]){1}', raw).group() #first sentence
    #fix!!!
    
    def get_title_and_first_sentence(self, qword):
        return self.engine.whats_new(qword)
    
    def top_entity_associated_with_item(self, item, weighted=True, rangee=5, most_c=10):
        #search for item.
            #for i in feed. if i == feed:
        #create a list of words that are close to word in proximity
        #score based on proximity to word.
        #documents is already a list
        word_freq = Counter()
        for i in self.ent_dict:
            for x in range(len(self.ent_dict[i])):
                if self.ent_dict[i][x] == item:
                    for z in range(rangee):
                        if not (x - z < 0):
                            if self.ent_dict[i][x-z] != self.ent_dict[i][x]:
                                if weighted:
                                    word_freq[self.ent_dict[i][x-z]] += 1/abs((x -(x-z)))
                                else:
                                    word_freq[self.ent_dict[i][x-z]] += 1
                        if not (x + z > (len(self.ent_dict[i]) - 1)):
                            if self.ent_dict[i][x+z] != self.ent_dict[i][x]:
                                if weighted:
                                    word_freq[self.ent_dict[i][x+z]] += 1/abs((x -(x+z)))
                                else:
                                    word_freq[self.ent_dict[i][x+z]] += 1
        return word_freq.most_common(most_c)
    
    def top_entity_dict(self, item, most_c=10):
        #documents is already a list
        #turn each list into a counter, add all counters together. 
        mega_counter = Counter()
        for i in self.ent_dict:
            #get list of counters etc
            if item in self.ent_dict[i]:
                c = Counter(self.ent_dict[i])
                del c[item]
                mega_counter += c
        return mega_counter.most_common(most_c)

In [13]:
edatb = entityDatabase("/Users/megankaye/Desktop/BeaverWorks/Work/NumberWon/numberwon/entity/testfeed.pickle")

In [14]:
#edatb.ent_dict

In [15]:
edatb.top_entity_associated_with_item("Katy Perry", weighted=True, rangee=100)

[('MTV Video Music Awards', 2.3559523809523806),
 ('Pop', 2.225),
 ('VMA', 0.7747326203208557),
 ('FILE', 0.6428571428571428),
 ('Britain', 0.6428571428571428),
 ('Perry', 0.5602564102564103),
 ('MTV', 0.5315734989648033),
 ('Pyramid Stage', 0.5),
 ('Glastonbury Festival', 0.5),
 ('Worthy Farm', 0.45)]

In [16]:
edatb.top_entity_dict("Katy Perry", most_c=10)

[('MTV Video Music Awards', 3),
 ('MTV', 2),
 ('Perry', 2),
 ('Pop', 2),
 ('VMA', 2),
 ('Calvin Harris', 1),
 ('Somerset', 1),
 ('Zayn Malik', 1),
 ('American Idol', 1),
 ('FILE', 1)]

In [21]:
edatb.docsearch("Trump")

'Stephen Colbert brings animated satire of Donald Trump to Showtime\n\nFILE PHOTO:'

In [22]:
edatb.engine.whats_new("Trump")

' Stephen Colbert brings animated satire of Donald Trump to Showtime\n\nFILE PHOTO: CBS television comedian Stephen Colbert rehearses a sketch on the floor of the Republican National Convention in Cleveland, Ohio, US July 17, 2016'